In [ ]:
# Get & unzip data
!rm -rf ../content/*
!wget https://graphicwg.irafm.osu.cz/storage/carvana.zip --no-check-certificate
!unzip -q carvana.zip
!rm carvana.zip

In [ ]:
# Install dependencies
!pip install segmentation_models

In [ ]:
# Set environment variable so segmentation_models uses correct Keras
%env SM_FRAMEWORK = tf.keras

# Import libraries
import segmentation_models as sm
import cv2
from glob import glob
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [5]:
# Create & compile model
model = sm.Unet("resnet18", classes=2, input_shape=(320, 480, 3), encoder_weights=None)
model.compile("Adam", sm.losses.DiceLoss())

In [ ]:
# Load data and labels into RAM
def load_data(data_folder, label_folder):
  data_paths = sorted(glob(data_folder + "*.*"))                                                               #  data_path/name.jpg
  data = [cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB) for path in tqdm(data_paths, desc="Loading data from '" + data_folder + "'")]

  labels_paths = [path.replace(data_folder, label_folder).replace(".jpg", "_mask.png") for path in data_paths] # label_path/name_mask.png
  labels = [cv2.imread(path) for path in tqdm(labels_paths, desc="Loading labels from '" + label_folder + "'")]

  return data, labels

x_train, y_train = load_data('./train/', './train_masks/')
x_valid, y_valid = load_data('./valid/', './valid_masks/')